# 동영상 분석
## 웹카메라의 이미지를 실시간으로 출력하기

In [36]:
import cv2
import numpy as np

# 웹카메라로부터 입력받기
cap = cv2.VideoCapture(0)
while True:
    #카메라로부터 이미지 읽기
    _, frame = cap.read()
    #이미지를 축소하기
    frame = cv2.resize(frame, (500,300))
    #윈도우에 이미지 출력하기
    cv2.imshow('OpenCV Web Camera - press ENTER OR ESC -> EXIT', frame)
    # ESC 또는 Enter키가 입력되면 반복 종료하기
    k = cv2.waitKey(1) # 1m sec 대기
    if k == 27 or k == 13: break
        
cap.release()
cv2.destroyAllWindows() # 윈도우 제거하기
    

## 카메라 이미지에서 붉은색 성분만 추출하기 

    frame 은 3차원 객체로 [:,:,0] 은 가장 처음 차원에 모든 요소를 선택후 두번째 차원의 모든 요소를 선택 
    그리고 가장 마지막 차원의 첫번째 요소들만 적용
    예제 > [[[1, 2, 3],[4, 5, 6],[7, 8, 9], ... [x, y, z] ]] -> 1, 4, 7 번 위치에 있는 요소(x위치)들만 0으로 적용함

In [19]:
import cv2
import numpy as np

# 웹카메라로부터 입력받기
cap = cv2.VideoCapture(0)
while True:
    #카메라로부터 이미지 읽기
    _, frame = cap.read()
    #이미지를 축소하기
    frame = cv2.resize(frame, (500,300))
    #파란색과 녹색 부분 제거하기
    frame[:,:,0] = 0
    frame[:,:,1] = 0
    #윈도우에 이미지 출력하기
    cv2.imshow('red Camera - press ENTER OR ESC -> EXIT', frame)
    # ESC 또는 Enter키가 입력되면 반복 종료하기
    k = cv2.waitKey(1) # 1m sec 대기
    if k == 27 or k == 13: break
        
cap.release()
cv2.destroyAllWindows() # 윈도우 제거하기

## HSV 색공간을 사용해 색 검출하기
### 위의 방법은 붉은 부분이 추출된거 같지는않다. 
### HSV 방법은 색상(Hue), 채도(Saturation), 명도(Value Brightness)라는 3가지 값을 사용해 색을 표현하는 방법

In [4]:
import cv2
import numpy as np

# 웹카메라로부터 입력받기
cap = cv2.VideoCapture(0)
while True:
    #카메라로부터 이미지 읽기
    _, frame = cap.read()
    #이미지를 축소하기
    frame = cv2.resize(frame, (500,300))
    #색공간을 HSV로 변환하기
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV_FULL)
    # 색분할하기
    h = hsv[:, :, 0]
    s = hsv[:, :, 1]
    v = hsv[:, :, 2]
    # 붉은색을 가진 요소만 출력하기
    img = np.zeros(h.shape, dtype=np.uint8)
    img[((h < 50) | (h > 200)) & (s > 100)] = 255
    
    #윈도우에 이미지 출력하기
    cv2.imshow('red Camera - press ENTER OR ESC -> EXIT', img)
    # ESC 또는 Enter키가 입력되면 반복 종료하기
    k = cv2.waitKey(1) # 1m sec 대기
    if k == 27 or k == 13: break
        
cap.release()
cv2.destroyAllWindows() # 윈도우 제거하기

# 화면에 움직임이 있는 부분 추출하기 - cv2.absdiff()

감시카메라와 같은 용도로 화면에서 큰 움직임이 있을경우 저장하는 방법으로 사용할수 있음

In [14]:
import cv2
import numpy as np

# 웹카메라로부터 입력받기
cap = cv2.VideoCapture(0)

img_last = None     # 이전 프레임을 저장할 변수
green = (0, 255, 0)


while True:
    #카메라로부터 이미지 읽기
    _, frame = cap.read()
    #이미지를 축소하기
    frame = cv2.resize(frame, (500,300))
    #흑백 이미지로 변환하기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9,9), 0)
    img_b = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)[1]
    #차이 확인하기
    if img_last is None:
        img_last = img_b
        continue
    frame_diff = cv2.absdiff(img_last, img_b)
    cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    #차이가 있는 부분 출력하기
    for pt in cnts:
        x, y, w, h = cv2.boundingRect(pt)
        if w < 50: continue # 작은 변경 무시하기
        cv2.rectangle(frame, (x, y), (x+w, y+h), green, 2)
    #프레임 변수에 저장해두기
    img_last = img_b
    
    #윈도우에 이미지 출력하기
    cv2.imshow('Diff Camera - press ENTER OR ESC -> EXIT', frame)
    cv2.imshow('Diff data', frame_diff)
    cv2.imshow('Diff data2', img_b)
    # ESC 또는 Enter키가 입력되면 반복 종료하기
    k = cv2.waitKey(1) # 1m sec 대기
    if k == 27 or k == 13: break
        
cap.release()
cv2.destroyAllWindows() # 윈도우 제거하기

# 이미지 파일 쓰기

In [22]:
import cv2
import numpy as np

#카메라 입력받기
cap = cv2.VideoCapture(0)
# 동영상 출력 전용 객체 생성하기
fmt = cv2.VideoWriter_fourcc("m", "p", "4", "v")
fps = 30.0
size = (1280, 640)
writer = cv2.VideoWriter("test.m4v", fmt, fps, size)

while True:
    _, frame = cap.read()
    # 이미지 축소하기
    frame = cv2.resize(frame, size)
    # 이미지 저장하기
    writer.write(frame)
    # 화면에도 출력하기
    cv2.imshow("frame", frame)
    # Enter 키가 입력되면 반복 종료하기
    if cv2.waitKey(1) ==13: break

writer.release()
cap.release()
cv2.destroyAllWindows()

# 동영상에 움직이는 부분 캡쳐해서 저장하기

In [35]:
import cv2, os

img_last = None # 이전 프레임을 저장할 변수
no = 0 # 이미지 장수
save_dir = "./exfish"
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

# 동영상 파일부터 입력받기
cap = cv2.VideoCapture("fish.mp4")

while True:
    # 이미지 추출하기
    is_ok, frame = cap.read()
    if not is_ok: break   # is_ok는 캡쳐한 비디오 프레임 없으면 False를 반환한다.
    frame = cv2.resize(frame, (640, 360))
    # 흑백 이미지로 변환하기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
    # 차이 확인하기
    if not img_last is None:
        frame_diff = cv2.absdiff(img_last, img_b)
        cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        # 차이가 있는 부분 파일로 출력하기
        for pt in cnts:
            x,y,w,h = cv2.boundingRect(pt)
            if w < 100 or w > 500: continue
            # 추출 영역 저장
            imgx = frame[y:y+h, x:x+w]
            outfile = save_dir+"/"+str(no)+".jpg"
            cv2.imwrite(outfile, imgx)
            no += 1
    img_last = img_b
cap.release()
print("ok")

ok


# 머신 러닝으로 동영상에서 열대어가 많이 나오는 부분 찾기 

In [17]:
import cv2
import os, glob
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [18]:
# 이미지 학습 크기와 경로 지정하기
img_size = (64, 32)
# path = os.path.dirname(os.path.abspath('3 OpenCV와 머신러닝 - 이미지, 동영상'))
# path_fish = path + r'\fish'
# path_nofish = path + r'\nofish'
path_fish = './fish'
path_nofish = './nofish'
x = []
y = []

In [19]:
# 이미지 데이터를 읽어 들이고 배열에 넣기
def read_dir(path, label):
    files = glob.glob(path+"/*.jpg")
    for f in files:
        img = cv2.imread(f)
        img = cv2.resize(img, img_size)
        img_data = img.reshape(-1, ) # 1차원으로 변환하기
        x.append(img_data)
        y.append(label)

In [20]:
# 이미지 데이터 읽어들이기
read_dir(path_nofish, 0)
read_dir(path_fish, 1)

In [21]:
# 데이터를 학습 전용과 테스트 전용으로 분리하기
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [22]:
# 데이터 학습하기
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

RandomForestClassifier()

In [23]:
# 정답률 확인하기
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9705882352941176


In [24]:
# 데이터 저장하기
joblib.dump(clf, 'fish.pkl')

['fish.pkl']

#  동영상 분석하기

In [25]:
import cv2, os, copy
import joblib

In [26]:
# 학습한 데이터 읽기
clf = joblib.load("fish.pkl")
output_dir = "./bestshot"
img_last = None
fish_th = 3
count = 0
frame_count = 0
if not os.path.isdir(output_dir): os.mkdir(output_dir)

In [28]:
# 동영상 파일 읽기
cap = cv2.VideoCapture("fish.mp4")
while True:
    # 이미지 추출하기
    is_ok, frame = cap.read()
    if not is_ok: break
    frame = cv2.resize(frame, (640, 360))
    frame2 = copy.copy(frame)
    frame_count += 1
    
    # 이전 프레임과 비교를 위해 흑백으로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
    if not img_last is None:
        # 차이 추출하기
        frame_diff = cv2.absdiff(img_last, img_b)
        cnts = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        # 차이가 있는 부분에 물고기가 있는지 확인하기
        fish_count = 0
        for pt in cnts:
            x, y, w, h = cv2.boundingRect(pt)
            if w< 100 or w > 500: continue # 노이즈 제거
            # 추출 영역에 물고기가 있는지 확인하기
            imgx = frame[y:y+w, x:x+h]
            imagex = cv2.resize(imgx, (64, 32))
            image_data = imagex.reshape(-1, )
            pred_y = clf.predict([image_data])
            if pred_y[0] == 1:
                fish_count += 1
                cv2.rectangle(frame2, (x, y), (x+w, y+h), (0, 255, 0), 2)
        # 물고기가 많이 있는지 확인하기 
        if fish_count > fish_th:
            fname = output_dir+"/fish"+str(count)+".jpg"
            cv2.imwrite(fname, frame)
            count += 1
    cv2.imshow("FISH!", frame2)
    if cv2.waitKey(1) == 13: break
    img_last = img_b
cap.release()
cv2.destroyAllWindows()
print("ok", count, "/", frame_count)

ok 24 / 2790
